<a href="https://colab.research.google.com/github/adasegroup/ML2020_seminars/blob/master/seminar9/feature_and_model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature selection

A good grasp of feature selection methods leads to better performing models, better understanding of the underlying structure and characteristics of the data and leads to better intuition about the algorithms that underlie many machine learning models.

There are in general two reasons why feature selection is used:
1. Reducing the number of features, to reduce overfitting and improve the generalization of models.
2. To gain a better understanding of the features and their relationship to the response variables.

In some cases a model may become more interpretable.

### Toy function


$$
y = \sin (4\pi x_1 x_2) + 4(x_3 - 0.5)^2 + x_4 + 0.5 x_5 + \varepsilon,
$$
where 

* $x_1, \ldots, x_5 \sim Uniform([0, 1])$ and $\varepsilon \sim \mathcal{N}(0, 1)$.
* We also add noise variables $x_6, x_7, x_8$
* and  $x_9, x_{10}, x_{11}$ strongly correlated with $x_2, x_3, x_4$ generated by $f(x) = x + \mathcal{N}(0, 025)$.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


def toy_function(x, sigma_noise=1, random_state=None):
    if random_state is None:
        random_state = np.random.RandomState()
    y = (np.sin(4 * np.pi * x[:, 0] * x[:, 1])
         + 4 * (x[:, 2] - 0.5)**2
         + x[:, 3]
         + 0.5 * x[:, 4]
         + random_state.normal(0, sigma_noise, size=(len(x)))
        )
    return y

Let's generate training, validation and test sets

In [ ]:
from sklearn.model_selection import train_test_split


random_state = np.random.RandomState(1)
size = 1500
X = random_state.rand(size, 11)
y = toy_function(X, sigma_noise=0.01, random_state=random_state) 

#Add 3 additional correlated variables (correlated with X1-X3)
X[:, 8:] = X[:, 1:4] + random_state.normal(0, .025, size=(size, 3))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=random_state)
print(X_train.shape, X_valid.shape, X_test.shape)

## F-test based feature importances

Consider two models, $M_1$ and $M_2$, where $M_1$ has $p_1$ parameters and $M_2$ has $p_2$ parameters,
and $p_2 > p_1$,
parameters of $M_1$ are subset of parameters of $M_2$.

* We want to check whether $M_2$ significantly better than $M_1$
* Let us consider the value
    $$
    F=\frac{\left({\frac {{\text{RSS}}_{1}-{\text{RSS}}_{2}}{p_{2}-p_{1}}}\right)}
           {\left({\frac {{\text{RSS}}_{2}}{n-p_{2}-1}}\right)},
    $$ 
    where $RSS_i$ is the Residual Sum of Squares of $M_i$.  

* If $M_2$ is not significantly better than $M_1$, $F$ will have an F-distribution, with $(p_2−p_1, n−p_2)$ degrees of freedom.
* We rejecte the hypothesis if the $F$ value is greater than the critical value of the F-distribution for some desired false-rejection probability (e.g. 0.05).


* Relies on $\mathbf{\color{orange}{\mbox{linearity}}}$ of models $M_1, M_2$

Now, let's apply F-test to the generated data and analyze results

In [ ]:
from sklearn.feature_selection import f_regression
import pandas as pd


feature_names = np.array(
    ['x1 [sin(x1 x2)]', 'x2 [sin(x1 x2)]', 'x3 [x3^2]', 'x4', r'$\frac12$x5',
     'noise1', 'noise2', 'noise3',
     'corr(x2)', 'corr(x3)', 'corr(x4)'])

feature_scores = pd.DataFrame(columns=['F test'], index=feature_names)

feature_scores['F test'], _ = f_regression(X_train, y_train)

feature_scores['F test'] /= feature_scores['F test'].max()
feature_scores.round(2)

**Conclusion**: F-test captures well only linear dependencies!

</br>

### Mutual Information

This method estimates how much information about the target variable $Y$ is contained in the feature $X$.  
To do this we calculate the value
$$
MI(X, Y) = H(Y) - H(Y | X) = \sum_{x, y} p(x, y) \log \frac{p(x, y)}{p(x)p(y)} = D_{KL}(p(x, y) \, ||  \, p(x)p(y)),
$$
where $p(x, y), p(x), p(y)$ are joint and marginal densities of random variables $X$ and $Y$.

* $MI(X, Y) = 0 \Leftrightarrow $ $X$ and $Y$ are independent
* Higher values $\rightarrow$ higher dependency
* Can capture $\mathbf{\color{green}{\mbox{non-linear}}}$ dependcies

#### Let's compare F-test and Mutual Information

In [ ]:
from sklearn.feature_selection import mutual_info_regression


method_name = 'Mutual Information'
feature_scores[method_name] = mutual_info_regression(X_train, y_train)

feature_scores[method_name] /= feature_scores[method_name].max()
feature_scores.round(2).style.background_gradient(cmap='Reds', high=1.01)

**Conclusion**: better estimate for non-linear dependencies

</br>

## Random Forest based feature importances
Random forests provide straightforward methods for feature ranking

* The importance of a feature is the (normalized) total reduction of the criterion brought by that feature:
    * For each split in a tree get how much it decreases Gini index.
    * Take sum of this values over all splits of the feature over all trees

In [ ]:
from sklearn.ensemble import RandomForestRegressor 

method_name = 'Random Forest'
random_forest = RandomForestRegressor(n_estimators=100, random_state=random_state)
random_forest.fit(X_train, y_train)

feature_scores[method_name] = random_forest.feature_importances_.reshape(-1, 1)
feature_scores[method_name] /= feature_scores[method_name].max()
feature_scores.round(2).style.background_gradient(cmap='Reds', high=1.01)

## L1 and L2 regularizations

Regularization adds penalty to a model in order to prevent overfitting and improve generalization.  

* Instead of minimizing a loss function $L(X,Y)$, we minimize $L(X,Y)+\alpha\|w\|$,
    * where $w$ is the vector of model coefficients,
    * $\|\cdot\|$ is typically L1 or L2 norm
    * $\alpha$ is a parameter, specifying the amount of regularization ($\alpha = 0$ implies no regularization).
* **Lasso** - linear model with L1 regularization.
* **RidgeRegression** - linear model with L2 regularization
* **Lasso** efficiently **zeros out** coefficients, **RidgeRegression** makes them smaller but typically non-zero.
* $\rightarrow$ Lasso can be used for feature selection

In [ ]:
from sklearn.linear_model import Lasso, Ridge

method_name = 'Lasso'
lasso = Lasso(alpha=0.001, normalize=True).fit(X_train, y_train)
feature_scores[method_name] = np.abs(lasso.coef_).reshape(-1, 1)
feature_scores[method_name] /= feature_scores[method_name].max()

method_name = 'Ridge'
ridge = Ridge(normalize=True).fit(X_train, y_train)
feature_scores[method_name] = np.abs(ridge.coef_).reshape(-1, 1)
feature_scores[method_name] /= feature_scores[method_name].max()

feature_scores.round(2).style.background_gradient(cmap='Reds', high=1.01)

### Task

What if we remove correlated component?

**Task**: 
1. Build `RidgeRegression` model using all features. Check coefficients for features `x4` and `corr(x4)`.
2. Build `RidgeRegression` model without the last features. Check coefficient for feature `x4`.
3. What can you conclude?

In [ ]:
# Your code here


</br>

## Forward-backward methods

The idea is to add or delete features and look how loss function or some other criteria changes.

Decision about adding or deleting a feature may be made based on:

- AIC
- BIC
- validation error
- Mallows $C_p$
- `model.score()`

**Question**: can we use error on the training set?

### Task. Implement forward method

Implement the following algorithm:

1. Start with empty list of features.
2. Add feature that increases model quality the most (use `model.score()` function)
3. Iterate step 2 until required number of features is selected


In [ ]:
def forward_steps(X, y, X_valid, y_valid, n_features, model):
    """
    Parameters
    ==========
    X : np.ndarray
        Training inputs

    y : np.ndarray
        Training targets

    X_valid : np.ndarray
        Validation inputs

    y_valid : np.ndarray
        validation targets

    n_features: int
        Number of features to select

    model : sklearn model
        
    Returns
    =======
    feature_list : list
        List of features
    """
    
    feature_list = []
    
    # Your code here
    
    # fill variable feat_list with
    # the selected features and return it
    
    return feature_list

In [ ]:
features_idx = forward_feat = forward_steps(X_train, y_train,
                                            X_valid, y_valid,
                                            5, random_forest)
print(feature_names[features_idx])

</br>

## Permutation based feature importance


**Idea**: measure how much the score (accuracy, F1, R^2, etc.) decreases when a feature is not available.

It can be done by:

- [ ] Removing feature from the dataset, re-training the estimator and check the score - can be $\mathbf{\color{orange}{\mbox{computationally intensive}}}$

- [x] Replace feature with random noise - now the column doesn't contain useful information.
  
  * Noise should be from the same distribution as original feature values (as otherwise estimator may fail).
  * -> $\mathbf{\color{blue}{\mbox{shuffle values for a feature}}}$, i.e. use other examples’ feature values

Some properties:
* Allows to estimate feature importance for **generalization** on new data set  
* If there are correlated features the importance of both can be low

### Task. Implement permutation importance

1. Using fitted model calculate its base_score - score, obtained using all features
2. Take one feature column and replace its values by the permuted values, calculate
   the score and subtract it from base_score, i.e. calculate `score_decrease`
3. Repeat step 2 several times and calculte mean `score_decrease`
4. Do steps 2 and 3 for each feature

In [ ]:
def _permutation_importance(model, X, y, 
                            feature, n_iters=5, random_state=None):
    """
    Calculate permutation importances for the given feature
    
    Parameters
    ==========
    model : sklearn model
        Model with `score()` method
        
    X : np.ndarray
        Training inputs
        
    y : np.ndarray
        Training targets
        
    feature : int
        Index of feature importance should be calculated for
        
    n_iters : int
        Number of permutation rounds
        
    Returns
    =======
    importance : np.ndarray
        Feature importances
    """
    if random_state is None:
        random_state = np.random.RandomState(1)
    if isinstance(random_state, int):
        random_state = np.random.RandomState(random_state)
    
    importance = None
    
    # Your code here

    return importance

def get_permutation_importance(model, X, y, n_iters=100, random_state=None):
    permutation_importances = np.zeros(X.shape[1])
    for i in range(X.shape[1]):
        permutation_importances[i] = _permutation_importance(model, X, y, i)
    return permutation_importances

In [ ]:
random_forest = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)
permutation_importances = get_permutation_importance(random_forest, X_valid, y_valid)

feature_scores['Permutation Importance'] = permutation_importances.reshape(-1, 1)
feature_scores['Permutation Importance'] /= feature_scores['Permutation Importance'].max()

feature_scores.round(2).style.background_gradient(cmap='Reds', high=1.01)

It is useful to visualize feature importances using bar plot

In [ ]:
def plot_feature_scores(X, scores):
    sorting_idx = scores.argsort()

    plt.figure(figsize=(10, 0.3 * len(scores)))
    plt.barh(range(len(scores)), scores[sorting_idx], height=0.5)
    plt.yticks(range(len(scores)), X.columns[sorting_idx])
    for i, v in enumerate(scores[sorting_idx]):

        if np.isfinite(v):
            v_str = '{:.3f}'.format(v) if v != int(v) else '{}'.format(int(v))
            plt.text(v + 0.005 * scores.max(), i - 0.22, v_str)
    plt.grid(True)
    plt.ylim([-0.5, len(scores) - 0.5])
    plt.show()

In [ ]:
plot_feature_scores(pd.DataFrame(X_train), permutation_importances)

### Use RFECV to get optimal number of features

To choose number of features automatically, RFECV module may be used.

In [ ]:
from sklearn.feature_selection import RFECV

# use RFECV to select features, use step = 50
rfe = RFECV(random_forest, n_jobs=-1, verbose=True, step=1, cv=3)
rfe.fit(X_train, y_train)

feature_scores['RFECV'] = X.shape[1] - rfe.ranking_.astype(float).reshape(-1, 1)
feature_scores['RFECV'] /= feature_scores['RFECV'].max()
feature_scores.round(2).style.background_gradient(cmap='Reds', high=1.01)

In [ ]:
# output selected number of features
print("Number selected features =", np.sum(rfe.support_))

### Task. Calculate quality of the model with the selected features


* Use RandomForestRegression
* Use test set and model's `score()` function to calculate model error
* Try F-test, RandomForest embedded feature importances, Permutation Importance and RFECV
* Select 5 features for each feature importance method

In [ ]:
def compare_methods(model, X_train, y_train, X_valid, y_valid, X_test, y_test):
    methods = {
        'F test': lambda X, y, X_valid, y_valid: f_regression(X, y)[0],

        'RandomForest': lambda X, y, X_valid, y_valid: model.fit(X, y).feature_importances_,

        'PermutationImportance': lambda X, y, X_valid, y_valid: get_permutation_importance(
            model.fit(X, y), X_valid, y_valid),

        'RFECV': lambda X, y, X_valid, y_valid: -RFECV(
            model, n_jobs=-1, step=1, cv=3).fit(X, y).ranking_
    }

    model_scores = pd.DataFrame(columns=list(methods.keys()))

    # Your code here
        
    return model_scores

Check the result

In [ ]:
compare_methods(random_forest, X_train, y_train, X_valid, y_valid, X_test, y_test)

results = [compare_methods(random_forest, X_train, y_train,
                           X_valid, y_valid, X_test, y_test)
           for _ in range(10)]

pd.concat(results, axis=0, ignore_index=True).describe()

### Akaike Information criterion (AIC)

Suppose that we have a statistical model of some data. Let $k$ be the number of estimated parameters in the model.
Let $\hat{L}$ be the maximum value of the likelihood function for the model.
Then the AIC value of the model is the following.

$$
\mathrm {AIC} = 2k - 2\ln(\hat{L})
$$


* We would like to minimize AIC
* AIC rewards goodness of fit (likelihood function)
* It includes a penalty - an increasing function of the number of parameters.

AIC is founded in information theory. Suppose that the data is generated by some unknown process $f$.
We consider two candidate models to represent $f$: $_1$ and $M_2$.
If we knew $f$, then we could find the information lost from using $M_1$ to represent $f$ by calculating the Kullback–Leibler divergence,
$DKL(f || M_1)$;
similarly, the information lost from using $M_2$ to represent $f$ could be found by calculating $DKL(f || M_2)$.
We would then choose the candidate model that minimized the information loss.

AIC - is an approximate value of how much more information is lost by $M_1$ than by $M_2$.

Valid only asymptotically

### Bayesian Information Criterion

Defined as 

$$
{\rm BIC} = \ln(N)k - 2\ln(\hat{L})
$$

* We would like to minimize BIC
* It can be shown that for model $M$ and given data set $D$ the posterior probability is approximately

$$
p(M | D) \propto \exp(-{\rm BIC}) p(M)
$$

### Task 4. Use `LassoLarsIC` to choose optimal alpha based on AIC and BIC

* Parameter $\alpha$ in LASSO affects the number of non-zero coefficients $\Rightarrow$ allows to control number of parameters in the model
* LARS algorithm for lasso for each possible number of non-zero coefficients estimates $\alpha$ that gives such number of non-zeros.
* $\Rightarrow$ using AIC or BIC we can choose $\alpha$.

**Task**:
1. choose $\alpha$ using AIC, print non-zero feature's names
2. choose $\alpha$ using BIC, print non-zero feature's names
3. Compare

In [ ]:
from sklearn.linear_model import LassoLarsIC

# Your code here
